In [1]:
import pickle
import time
import DataFormatterEdit
import cknnEdit
import pandas as pd
#import importlib
#importlib.reload()
#from IPython.display import display
from utils import build_graph, split_validation
from utils import Data

from model import *

In [2]:
import torch
import numpy as np
from tqdm.notebook import tqdm

In [3]:
def make_pd_data_frame(train_data, session_key='session_id', item_key='item_id', time_key='eventdate'):
    df = pd.DataFrame({session_key:[], item_key:[], time_key:[]})
    ses_ids = []
    ses_dates = []
    ses_seqs = []
    for ses_id, ses_date, ses_seq in tqdm(zip(train_data[0], train_data[1], train_data[2]), total=len(train_data[0])):
        for ses_ in ses_seq:
            ses_date_ = int(ses_date.astype(np.float))
            ses_ids.append(int(ses_id))
            ses_dates.append(ses_date_)
            ses_seqs.append(ses_)
    ses_ids = np.asarray(ses_ids)
    ses_dates = np.asarray(ses_dates)
    ses_seqs = np.asarray(ses_seqs)
    d = np.concatenate((ses_ids,ses_seqs,ses_dates))
    d = np.reshape(d,(3,-1))
    d = d.T
    column_names = [session_key, item_key, time_key]
    df = pd.DataFrame(d, columns=column_names)
    return df

In [4]:
def scr_conv(scr):
    ret = []
    for i in tqdm(scr):
        for j in i:
            arr = np.zeros((n_item,1))
            for idx, val in zip(list(j.keys()), list(j.values())):
                arr[int(idx)-1] = val
            ret.append(arr)
    ret = np.array(ret)
    return ret

In [5]:
def num_data(data, str_):
    values = data.loc[:,[str_]].values
    values = values.reshape(-1,)
    new = np.unique(values)
    return new.shape[0]

In [6]:
fold = 5
_path = 'amz_book_overall_reflect_40_000.csv'#(58.92597968069666, 44.82286867076933)
_path = 'amz_book_40_000.csv'#(60.9504132231405, 50.03743392333061)
_path = 'amz_elec_40_000.csv'#(25.64450474898236, 17.445641008879555)
_path = 'amz_movi_40_000.csv'#(29.68591691995947, 18.977850405947404)
_path = 'amz_prep_book_book_5_20_s5_s20.csv'#n_item 616
_path = 'amz_prep_book_book_5_20_s5.csv'

In [7]:
data = pd.read_csv(_path, delimiter=';')
n_item = num_data(data, 'item_id')
print('n_item',n_item)
print('n_session',num_data(data, 'session_id'))

n_item 75607
n_session 37276


In [8]:
import importlib
importlib.reload(DataFormatterEdit)

<module 'DataFormatterEdit' from '/home/workplace/SR-GNN/termP/DataFormatterEdit.py'>

In [9]:
%%time
k_split_data_list, sess_clicks = DataFormatterEdit.get_k(_path, fold)
#train_data_preproc, test_data_preproc, train_data_, _ = DataFormatterEdit.get(filepath=_path)

CPU times: user 3.16 s, sys: 233 ms, total: 3.39 s
Wall time: 3.37 s


In [11]:
final_hit, final_mrr = [], []

for fold_ in range(fold):
    print('fold: ', fold_+1, '/', fold)
    test_data_preproc = k_split_data_list[fold_]
    train_data_preproc = k_split_data_list.copy()
    del train_data_preproc[fold_]
    train_data_ = _ = train_data = test_data = df = knn = mrr_20 = p_20 = scr = scr_convd = slices = None
    
    train_data_preproc, test_data_preproc, train_data_, _ = DataFormatterEdit.make_tr_ts_data_set(train_data_preproc, 
                                                                                                  test_data_preproc, 
                                                                                                  sess_clicks)
    
    train_data = Data(train_data_preproc, shuffle=True)
    test_data = Data(test_data_preproc, shuffle=False)
    #test_data.inputs.shape
    #train_data_preproc = [[subset],[ans_items]]
    #train_data_ = [[session_id],[session_date],[item_seq]]
    #train_data = obj
    n_node = n_item+1

    model = trans_to_cuda(SessionGraph(n_node))

    best_result = [0, 0]
    best_epoch = [0, 0]
    bad_counter = 0
    for epoch in range(20):
        print('-------------------------------------------------------')
        print('epoch: ', epoch)
        hit, mrr = train_test(model, train_data, test_data)
        flag = 0
        if hit >= best_result[0]:
            best_result[0] = hit
            best_epoch[0] = epoch
            flag = 1
        if mrr >= best_result[1]:
            best_result[1] = mrr
            best_epoch[1] = epoch
            flag = 1
        print('Best Result:')
        print(f'current hit: {hit}, mrr: {mrr}')
        print('\tPrecision@20:\t%.4f\tMRR@20:\t%.4f\tEpoch:\t%d,\t%d'% (best_result[0], best_result[1], best_epoch[0], best_epoch[1]))
        bad_counter += 1 - flag
        if bad_counter >= 10:
            break
    print('-------------------------------------------------------')

    df = make_pd_data_frame(train_data_)
    print(df)

    knn = cknnEdit.ContextKNN(data = df, session_key='session_id', item_key='item_id', time_key='eventdate')

    mrr_20, p_20, scr = knn.multi_predict_ts_data(test_data, 12)
    print('knn: p_20, mrr_20', p_20, mrr_20)

    scr_convd = scr_conv(scr)
    scr_convd.reshape((-1, n_item))
    scr_convd = np.reshape(scr_convd, (scr_convd.shape[0], scr_convd.shape[1],))

    model.eval()
    hit, mrr = [], []
    slices = test_data.generate_batch(model.batch_size)

    for i in tqdm(slices):
        targets, scores = forward(model, i, test_data)
        scores = trans_to_cpu(scores).detach().numpy()
        scores = np.add(scr_convd[i,:], scores)
        scores = torch.Tensor(scores)
        sub_scores = scores.topk(20)[1]
        sub_scores = trans_to_cpu(sub_scores).detach().numpy()
        for score, target, mask in zip(sub_scores, targets, test_data.mask):
            hit.append(np.isin(target - 1, score))
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
    final_hit.append(np.mean(hit)*100)
    final_mrr.append(np.mean(mrr)*100)
    print('epoch result: ',np.mean(hit)*100, np.mean(mrr)*100)
print('final result: ',np.mean(final_hit), np.mean(final_mrr))

fold:  1 / 5


100%|██████████| 7447/7447 [00:00<00:00, 28402.41it/s]
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


-------------------------------------------------------
epoch:  0


  1%|▏         | 2/149 [00:00<00:13, 10.78it/s]

	Loss:	57243.184


...

  0%|          | 0/7449 [00:00<?, ?it/s]

knn: p_20, mrr_20 9.021345146999597 2.6642071167849557


  0%|          | 0/7449 [00:00<?, ?it/s]

  0%|          | 0/149 [00:00<?, ?it/s]

epoch result:  9.518056114914755 2.195826880839767
epoch result:  9.347682920995627 2.265129879616548


In [12]:
final_hit, final_mrr

([9.52061232711159,
  9.664429530201343,
  9.012760241773002,
  9.022556390977442,
  9.518056114914755],
 [2.303353910411171,
  2.38367518147787,
  2.0942893432492076,
  2.348504082104724,
  2.195826880839767])